In [1]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter

In [8]:
def get_img_names(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines



def generate_col_names(size, dim = 2):
    """Generate the column names in this pattern
        [x1, y1, x2, y2 ...., x(size-1), y(size-1), pitch, yaw, ....]
        
        Inputs: 
            size -> No of landmarks 
            dim -> Default is 2 (x and y coordinates only),
                    if dim = 3 then z will be included
        
        outputs:
            Returns a list contains all the names
    """
    
    cols_names = []
    
    for i in range(size):
        cols_names.append(f'x{i+1}')
        cols_names.append(f'y{i+1}')
    
        if dim == 3:
            cols_names.append(f'z{i+1}')
        

    cols_names.extend(['pitch', 'yaw', 'roll',  'tdx', 'tdy', 'tdz', 'scale_factor'])
    
    return cols_names


def get_data(imgs, col_names, dim=2):
    """Get landamarks and pose parameters for each image
        
        Inputs:
            imgs -> list of all image names without  extensions
            col_names -> list of all parameter names
            dime -> Default is 2 (x and y coordinates only),
                    if dim = 3 then z will be included
        
        Output:
            Pandas DataFrame that contains landamarks and pose parameters for each image
    """
    
    # each element will contain all image parametars
    # 68 landmarks + pose parameters
    data = []

    for image_name in imgs:
        # image relative path
        image = f'./AFLW2000/{image_name}.jpg'

        # load mat file
        mat = sio.loadmat(f'./AFLW2000/{image_name}.mat')

        # Get 3d landmarks - 68 points
        pt3d = mat['pt3d_68']

        # Get Pose Parameters 
        # [pitch (phi), yaw (gamma), roll(theta), tdx, tdy, tdz, scale_factor]
        pose_params = mat['Pose_Para'][0]

        # Concatenating landmarks (xy coordinates) with pose params
        landmarks_xy_angles = np.hstack((pt3d.T[:, :dim].ravel(), pose_params))
        data.append(landmarks_xy_angles)
        
        # Coverer data to a dataframe
        data_df = pd.DataFrame(np.array(data))
        
        # rename Column names
        data_df.columns = col_names

    return data_df



In [55]:
# image names
img_names = get_img_names('./AFLW2000/*.mat')

# landmark size
size = 68

In [41]:
# column names
col_names_xy = generate_col_names(size, dim = 2)

# final data
data_xy = get_data(img_names, col_names_xy, 2)

In [56]:
data_xy

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y67,x68,y68,pitch,yaw,roll,tdx,tdy,tdz,scale_factor
0,121.868034,167.164215,122.367607,197.562622,126.819237,225.469650,130.831787,250.714905,137.523132,280.127869,...,330.701965,201.823303,324.903870,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,281.238159,265.079346,277.339417,287.044556,274.876953,307.054138,269.523773,324.187012,257.788269,340.334015,...,300.598602,197.534470,302.286499,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
2,236.385101,239.682129,239.198257,259.599121,242.566376,277.087280,243.971375,294.202820,242.828186,314.587280,...,304.681732,215.192352,304.631042,-0.184650,0.881137,-0.236852,301.050323,200.051437,-48.590317,0.001041
3,168.029221,256.937866,177.697510,277.588013,190.641602,295.489624,201.395294,311.751831,211.957214,329.989929,...,319.498596,232.172958,320.843994,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
4,280.462250,236.189209,287.249817,259.100372,293.892456,279.609741,297.539368,297.528564,295.677002,315.631744,...,312.920746,209.941147,312.058350,-0.882169,1.198004,-1.033374,323.511200,218.552780,-21.119057,0.001247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,112.578934,195.496368,113.787857,226.176926,119.259262,254.743179,125.335999,278.632111,133.637939,303.521118,...,324.518494,209.768448,322.085815,-0.197102,-0.070430,0.105118,217.692688,226.295181,-114.195282,0.001546
1996,111.117966,243.295334,113.647079,268.562378,115.328712,291.563843,120.565918,312.712769,134.109467,335.516144,...,317.922302,241.842422,318.168457,-0.232617,-1.418751,0.175960,103.658447,191.368881,-5.541234,0.001287
1997,308.227417,235.568451,309.905334,260.817841,311.432251,283.916595,310.488586,304.592224,303.662415,326.799042,...,315.051239,211.824646,313.333374,-1.447955,1.431267,-1.509418,336.144867,204.937408,-20.417292,0.001289
1998,106.611572,208.572235,106.089981,237.551620,107.328842,264.500000,109.023941,287.978760,116.216049,314.026215,...,313.581848,235.733337,312.269653,-0.420465,-1.191176,0.451515,110.517296,219.579651,-31.363331,0.001396


In [59]:
# column names
col_names_xyz = generate_col_names(size, dim = 3)

# final data
data_xyz = get_data(img_names, col_names_xyz, 3)

In [60]:
data_xyz

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,x68,y68,z68,pitch,yaw,roll,tdx,tdy,tdz,scale_factor
0,121.868034,167.164215,-65.108887,122.367607,197.562622,-73.032082,126.819237,225.469650,-81.247864,130.831787,...,201.823303,324.903870,9.237297,-0.399231,0.018227,0.085676,231.465317,262.066589,-102.793602,0.001587
1,281.238159,265.079346,-120.258255,277.339417,287.044556,-113.066238,274.876953,307.054138,-105.202911,269.523773,...,197.534470,302.286499,0.852242,0.470065,1.189533,0.300959,330.819702,173.194244,-44.367504,0.001181
2,236.385101,239.682129,-96.273857,239.198257,259.599121,-93.754654,242.566376,277.087280,-89.725311,243.971375,...,215.192352,304.631042,18.613464,-0.184650,0.881137,-0.236852,301.050323,200.051437,-48.590317,0.001041
3,168.029221,256.937866,-111.598221,177.697510,277.588013,-107.077736,190.641602,295.489624,-102.609352,201.395294,...,232.172958,320.843994,34.102440,-0.175379,0.299208,-0.373374,262.805054,211.201294,-102.680016,0.001294
4,280.462250,236.189209,-106.988503,287.249817,259.100372,-101.327782,293.892456,279.609741,-93.347626,297.539368,...,209.941147,312.058350,-8.034685,-0.882169,1.198004,-1.033374,323.511200,218.552780,-21.119057,0.001247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,112.578934,195.496368,-71.514404,113.787857,226.176926,-73.480286,119.259262,254.743179,-76.440567,125.335999,...,209.768448,322.085815,51.865341,-0.197102,-0.070430,0.105118,217.692688,226.295181,-114.195282,0.001546
1996,111.117966,243.295334,95.023521,113.647079,268.562378,90.800293,115.328712,291.563843,84.510887,120.565918,...,241.842422,318.168457,12.531554,-0.232617,-1.418751,0.175960,103.658447,191.368881,-5.541234,0.001287
1997,308.227417,235.568451,-115.265503,309.905334,260.817841,-111.355103,311.432251,283.916595,-104.058212,310.488586,...,211.824646,313.333374,-32.042725,-1.447955,1.431267,-1.509418,336.144867,204.937408,-20.417292,0.001289
1998,106.611572,208.572235,75.812988,106.089981,237.551620,75.338287,107.328842,264.500000,71.760956,109.023941,...,235.733337,312.269653,27.291553,-0.420465,-1.191176,0.451515,110.517296,219.579651,-31.363331,0.001396


In [72]:
imgs[8]

'image00020'

In [78]:
data_xyz.iloc[8, -7:]

pitch            -0.170780
yaw               0.383899
roll             -0.436852
tdx             267.145325
tdy             212.553406
tdz             -97.317200
scale_factor      0.001287
Name: 8, dtype: float64

In [100]:
phi, gamma, theta, tdx, tdy, tdz, scale_factor = data_xyz.iloc[8, -7:].to_numpy()

In [103]:
# R_x = [1 0 0 ; 0 cos(phi) sin(phi); 0 -sin(phi) cos(phi)];
# R_y = [cos(gamma) 0 -sin(gamma); 0 1 0; sin(gamma) 0 cos(gamma)];
# R_z = [cos(theta) sin(theta) 0; -sin(theta) cos(theta) 0; 0 0 1];

# phi = angle_x;
# gamma = angle_y;
# theta = angle_z;

R_x = np.array([[1, 0, 0],
               [0, cos(phi), sin(phi)],
               [0, -sin(phi), cos(phi)]])

R_y = np.array([[cos(gamma), 0, -sin(gamma)],
               [0, 1, 0],
               [sin(gamma), 0, cos(gamma)]])

R_z = np.array([[cos(theta), sin(theta), 0],
               [-sin(theta), cos(theta), 0],
               [0, 0, 1]])

R = R_x @ R_y @ R_z;


In [104]:
R

array([[ 0.8401349 , -0.39229351, -0.37453859],
       [ 0.35925913,  0.91983754, -0.15758099],
       [ 0.40633266, -0.00216712,  0.91372265]])

In [81]:
P = np.array([[1, 0, 0],
              [0, 1, 0],
             [0, 0, 1]])

t3d = np.array([tdx, tdy, tdz])

f = scale_factor

In [86]:
ProjectVertex = f * R @ P

In [89]:
t3d

array([267.14532471, 212.55340576, -97.31719971])

In [93]:
x1, y1, z1 = ProjectVertex[:, 0] + t3d 
x2, y2, z3 = ProjectVertex[:, 0] + t3d 
x3, y3, z3 = ProjectVertex[:, 0] + t3d 

In [121]:
image = f'./AFLW2000/image00013.jpg'
frame = cv2.imread(image)    

i = 5
nosex, nosey, nosez = data_xyz[['x30', 'y30', 'z30']].iloc[i, :].to_numpy()
pitch, yaw, roll, tdx, tdy, tdz, scale_factor = data_xyz.iloc[i, -7:].to_numpy()

pitch = pitch * np.pi / 180
yaw = -(yaw * np.pi / 180)
roll = roll * np.pi / 180

# X-Axis pointing to right. drawn in red
x1 = 100 * (cos(yaw) * cos(roll)) + tdx
y1 = 100 * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy
print(x1, y1)


 # Y-Axis | drawn in green
#        v
x2 = 100 * (-cos(yaw) * sin(roll)) + tdx
y2 = 100 * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

# Z-Axis (out of the screen) drawn in blue
x3 = 100 * (sin(yaw)) + tdx
y3 = 100 * (-cos(yaw) * sin(pitch)) + tdy


# cv2.circle(frame, (int(x1),int(y1)), radius=0, color=(255,55,50), thickness=15)
# cv2.circle(frame, (int(x2),int(y2)), radius=0, color=(55,55,50), thickness=15)
# cv2.circle(frame, (int(x3),int(y3)), radius=0, color=(5,55,50), thickness=15)
cv2.line(frame, (int(nosex), int(nosey)), (int(x1),int(y1)),(0,0,255),3)
cv2.line(frame, (int(nosex), int(nosey)), (int(x2),int(y2)),(0,0,255),3)
cv2.line(frame, (int(nosex), int(nosey)), (int(x3),int(y3)),(0,0,255),3)

cv2.imshow("wrf", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


325.6238594737466 199.49374319267432
